## HTorch hyperbolic embedding for the WordNet Mammals

In [1]:
import timeit
import torch, HTorch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
import logging
from hype.graph import load_edge_list, eval_reconstruction
from HTorch.layers import HEmbedding
from HTorch.optimizers import RiemannianSGD, RiemannianAdam
import sys, os, random
import json
import torch.multiprocessing as mp
from hype.graph_dataset import BatchedDataset

/home/ty367/anaconda3/envs/hy-torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# model defined using HTorch
class EnergyFunction(torch.nn.Module):
    def __init__(self, size, dim, sparse=False, manifold='PoincareBall', curvature=-1.0, **kwargs):
        super().__init__()
        # initialize layer, weights are automatically initialized around origin
        self.lt = HEmbedding(size, dim, sparse=sparse, manifold=manifold, curvature=curvature) 
        self.nobjects = size

    def forward(self, inputs):
        e = self.lt(inputs)
        with torch.no_grad():
            e.proj_()
        o = e.narrow(1, 1, e.size(1) - 1)
        s = e.narrow(1, 0, 1).expand_as(o)
        return o.Hdist(s).squeeze(-1)
    
    def loss(self, inp, target, **kwargs):
        return F.cross_entropy(inp.neg(), target)

In [3]:
# set meta-parameters, float precision etc.
os.environ["NUMEXPR_MAX_THREADS"] = '8'

def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# note d16, d32 may produce infs and NaNs due to imprecision
d16 = torch.float16; d32 = torch.float32; d64 = torch.float64
cpu = torch.device("cpu"); gpu = torch.device(type='cuda', index=0)
device = cpu
opt_dtype = d64

if opt_dtype == d16:
    dtype = "d16"
    torch.set_default_tensor_type('torch.HalfTensor')
elif opt_dtype == d32:
    dtype = "d32"
    torch.set_default_tensor_type('torch.FloatTensor')
else:
    dtype = "d64"
    torch.set_default_tensor_type('torch.DoubleTensor')
    
torch.manual_seed(42)
np.random.seed(42)

### Hyperparameters for PyTorch Poincare Halfspace model: (now use opt_epochs = 20)

In [4]:
## parameters; these are global in the notebook!
opt_maxnorm = 500000; opt_debug = False;
opt_dim = 2;
opt_negs = 50;  opt_eval_each = 20;
opt_sparse = True; opt_ndproc = 1;  opt_burnin = 20;
opt_dampening = 0.75; opt_neg_multiplier = 1.0; 
opt_burnin_multiplier = 0.01; 
###########################################################
opt_epochs = 2000; opt_batchsize = 32; 
opt_lr = 2.0;  opt_dscale = 0.3
#opt_manifold = "PoincareBall"
# opt_manifold = "Lorentz"
opt_manifold = "HalfSpace"
opt_curvature = -1.0
opt_task = 'mammals'
#######################################
FILE_NAME = "_".join([opt_task, 'lr', str(opt_lr), 'batch', str(opt_batchsize),
                      str(opt_epochs), "torch", dtype, str(opt_dscale)])

### Initializing logging and data loading

In [5]:
log_level = logging.DEBUG if opt_debug else logging.INFO
log = logging.getLogger('Embedding')
logging.basicConfig(level=log_level, format='%(message)s', stream=sys.stdout)
log.info('Using edge list dataloader')
idx, objects, weights = load_edge_list("wordnet/mammal_closure.csv", False) 
#idx, objects, weights = load_edge_list("/home/jl3789/Hyperbolic_Library/applications/poincare_embedding/wordnet/mammal_closure.csv", False) 

Using edge list dataloader


### Initializing model

In [6]:
def init_model(manifold, curvature, dim, idx, objects, weights, sparse=True):
    model_name = '%s_dim%d'
    mname = model_name % (manifold, dim)
    data = BatchedDataset(idx, objects, weights, opt_negs, opt_batchsize,
        opt_ndproc, opt_burnin > 0, opt_dampening)
    model = EnergyFunction(len(data.objects), opt_dim, sparse=sparse, manifold=manifold, curvature=curvature)
    data.objects = objects
    return model, data, mname

def adj_matrix(data):
    adj = {}
    for inputs, _ in data:
        for row in inputs:
            x = row[0].item()
            y = row[1].item()
            if x in adj:
                adj[x].add(y)
            else:
                adj[x] = {y}
    return adj

### Training

In [7]:
def data_loader_lr(data, epoch, progress = False):
    data.burnin = False 
    lr = opt_lr
    if epoch < opt_burnin:
        data.burnin = True
        lr = opt_lr * train._lr_multiplier
    loader_iter = tqdm(data) if progress else data
    return loader_iter, lr

In [8]:
def train(device, model, data, optimizer, progress=False):
    epoch_loss = torch.Tensor(len(data))
    LOSS = np.zeros(opt_epochs)
    for epoch in range(opt_epochs):
        epoch_loss.fill_(0)
        t_start = timeit.default_timer()
        # handling burnin, get loader_iter and learning rate
        loader_iter, lr = data_loader_lr(data, epoch, progress=progress)
        for i_batch, (inputs, targets) in enumerate(loader_iter):
            elapsed = timeit.default_timer() - t_start
            inputs = inputs.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
            preds = model(inputs) * opt_dscale
            loss = model.loss(preds, targets, size_average=True)
            loss.backward()
            optimizer.step(lr=lr)
            epoch_loss[i_batch] = loss.cpu().item()
            with torch.no_grad():
                loss = model.loss(preds.to(d64), targets, size_average=True)
                epoch_loss[i_batch] = loss.cpu().item()

        LOSS[epoch] = torch.mean(epoch_loss).to(d64).item()
        # since only one thread is used:
        log.info('json_stats: {' f'"epoch": {epoch}, '
                 f'"elapsed": {elapsed}, ' f'"loss": {LOSS[epoch]}, ' '}')
    return LOSS


# Training embedding

In [9]:
# setup model
seed_everything(1)
model, data, model_name = init_model(opt_manifold, opt_curvature, opt_dim, idx, objects, weights, sparse=opt_sparse)
data.neg_multiplier = opt_neg_multiplier
train._lr_multiplier = opt_burnin_multiplier
model = model.to(device)
print('the total dimension', model.lt.weight.data.size(-1))
print(">>>>>> # Tensor# | dtype is:", model.lt.weight.dtype, "| device is:", model.lt.weight.device)
# setup optimizer, both works, though a small lr should be used for RiemannianAdam (which is not tuned yet)
#optimizer = RiemannianAdam(model.parameters(), lr=opt_lr)
optimizer = RiemannianSGD(model.parameters(), lr=opt_lr)
# get adjacency matrix
adj = adj_matrix(data)
# begin training
start_time = timeit.default_timer()
loss = train(device, model, data, optimizer, progress=False)
train_time = timeit.default_timer() - start_time
print("Total training time is:", train_time)

<ipython-input-6-d3260de9c559>:4: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = BatchedDataset(idx, objects, weights, opt_negs, opt_batchsize,


the total dimension 2
>>>>>> # Tensor# | dtype is: torch.float64 | device is: cpu
json_stats: {"epoch": 0, "elapsed": 0.6585964319983759, "loss": 3.9316851160641013, }
json_stats: {"epoch": 1, "elapsed": 0.39639782100130105, "loss": 3.931278712296638, }
json_stats: {"epoch": 2, "elapsed": 0.3871085139999195, "loss": 3.9308722281065047, }
json_stats: {"epoch": 3, "elapsed": 0.3794079260005674, "loss": 3.930462608711671, }
json_stats: {"epoch": 4, "elapsed": 0.38465845199971227, "loss": 3.930052292162785, }
json_stats: {"epoch": 5, "elapsed": 0.38997425100023975, "loss": 3.9296411481076037, }
json_stats: {"epoch": 6, "elapsed": 0.4016620009988401, "loss": 3.9292304277919774, }
json_stats: {"epoch": 7, "elapsed": 0.45824854300008155, "loss": 3.9288169346744213, }
json_stats: {"epoch": 8, "elapsed": 0.3738608210005623, "loss": 3.928396309022339, }
json_stats: {"epoch": 9, "elapsed": 0.3923560799994448, "loss": 3.927976680180398, }
json_stats: {"epoch": 10, "elapsed": 0.3928097650004929, "l

# Evaluate embedding

In [10]:
class RES():
    """for logging results"""
    def __init__(self, loss, eval_res, weight):
        self.loss = torch.tensor(loss, dtype=torch.float64, 
                                 device=cpu)
        self.eval_res = torch.tensor(eval_res, dtype=torch.float64, 
                                     device=cpu)
        self.weight = weight

In [11]:
model_weight = model.lt.weight.clone()
# eval meanrank, maprank in the original model
meanrank, maprank = eval_reconstruction(adj, model_weight, workers=opt_ndproc)
if opt_manifold != "PoincareBall":
    # change to PoincareBall to derive the sqnorms metric
    model_weight = model_weight.to_other_manifold("PoincareBall")
sqnorms = torch.sqrt(torch.sum(torch.pow(model_weight, 2), dim=-1))
sqnorm_min = sqnorms.min().item()
sqnorm_avg = sqnorms.mean().item()
sqnorm_max = sqnorms.max().item()
eval_res = [meanrank, maprank, sqnorm_min, sqnorm_avg, sqnorm_max, train_time]
RESULTS = RES(loss, eval_res, model_weight)
# torch.save(RESULTS, "./results_weights/"+FILE_NAME+"_seed1"+ ".pt")
log.info(
    'json_stats final test: \n{'
    f'"sqnorm_min": {round(sqnorm_min,6)}, '
    f'"sqnorm_avg": {round(sqnorm_avg,6)}, '
    f'"sqnorm_max": {round(sqnorm_max,6)}, \n'
    f'"mean_rank": {round(meanrank,6)}, '
    f'"map": {round(maprank,6)}, '
    '}'
)

json_stats final test: 
{"sqnorm_min": 0.075631, "sqnorm_avg": 0.99849, "sqnorm_max": 1.0, 
"mean_rank": 1.150306, "map": 0.967294, }
